In [ ]:
# -*- coding: utf-8 -*-
import os
import warnings
from typing import List, Tuple, Dict, Optional

import numpy as np
import pandas as pd
from geopy.distance import geodesic
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tools.sm_exceptions import ConvergenceWarning
import statsmodels.api as sm

warnings.simplefilter("ignore", ConvergenceWarning)

# =============================
# Configurações
# =============================
STATIONS_CSV = "../data/all_stations.csv"
UNI_CSV = "./universidades_federais_coords_26_estados.csv"
CLEANED_BASE = "../data/cleaned_data"
YEARS = [2019, 2020, 2021, 2022, 2023, 2024]
RADIUS_KM = 35.0

FORECAST_PARAMETER = "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)"
EXOG_VARS = [
    "RADIACAO GLOBAL (KJ/m²)",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",
    "UMIDADE RELATIVA DO AR, HORARIA (%)",
]

RESAMPLE_RULE = "D"     # média diária
SPLIT_TRAIN = 0.80      # 80/20 treino/teste
ARIMA_ORDER = (3, 0, 1) # ARIMA(3,0,1)
FOURIER_PERIOD = 365    # dias
FOURIER_K = 4           # nº de harmônicos (gera 2*K colunas)

# =============================
# Utilidades básicas
# =============================
def calculate_distance_from_point_to_station(row, given_point_coord):
    station_coord = (row["LATITUDE:"], row["LONGITUDE:"])
    return geodesic(station_coord, given_point_coord).kilometers

def smape(y_true, y_pred):
    y_true, y_pred = pd.Series(y_true), pd.Series(y_pred)
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true)))

def create_fourier_terms(t, period, num_terms):
    terms = []
    for i in range(1, num_terms + 1):
        terms.append(np.sin(2 * np.pi * i * t / period))
        terms.append(np.cos(2 * np.pi * i * t / period))
    return np.column_stack(terms)

# =============================
# Leitura de estações e dados
# =============================
def load_all_stations() -> pd.DataFrame:
    df_all_stations = pd.read_csv(STATIONS_CSV, decimal=",", sep=";")
    df_all_stations.columns = [c.strip() for c in df_all_stations.columns]
    return df_all_stations

def load_weather_for_station_filename(filename_2019: str) -> Optional[pd.DataFrame]:
    """
    Usa o nome-base de 2019 e monta os caminhos 2019–2024.
    Concatena o que existir, e corrige 'RADIACAO GLOBAL (Kj/m²)' -> 'RADIACAO GLOBAL (KJ/m²)'.
    """
    dfs = []
    for year in YEARS:
        year_dir = f"{year}_cleaned"
        target = filename_2019.replace("2019", str(year))
        path = os.path.join(CLEANED_BASE, year_dir, target)
        if os.path.exists(path):
            try:
                df_weather_data = pd.read_csv(path, decimal=".", sep=";")
                # 🔧 correção de nomenclatura (Kj → KJ)
                if 'RADIACAO GLOBAL (Kj/m²)' in df_weather_data.columns:
                    df_weather_data.rename(
                        columns={'RADIACAO GLOBAL (Kj/m²)': 'RADIACAO GLOBAL (KJ/m²)'},
                        inplace=True
                    )
                dfs.append(df_weather_data)
            except Exception as e:
                print(f"[WARN] Falha ao ler {path}: {e}")
    if not dfs:
        return None
    return pd.concat(dfs, ignore_index=True)

def build_daily_panel_from_stations(df_nearest_stations: pd.DataFrame) -> Optional[pd.DataFrame]:
    """
    Retorna um DataFrame diário (index=Data) com média das variáveis numéricas
    das estações do raio. Mantém target + exógenas, faz ffill().
    """
    collected = []
    for filename in df_nearest_stations["Arquivo"]:
        dfw = load_weather_for_station_filename(filename)
        if dfw is not None:
            collected.append(dfw)
    if not collected:
        return None

    df = pd.concat(collected, ignore_index=True)
    if "Hora UTC" in df.columns:
        df = df.drop(columns=["Hora UTC"])
    df["Data"] = pd.to_datetime(df["Data"], errors="coerce")
    df = df.dropna(subset=["Data"]).sort_values("Data").set_index("Data")

    daily = df.resample(RESAMPLE_RULE).mean(numeric_only=True)

    # garantir colunas requeridas
    needed = [FORECAST_PARAMETER] + EXOG_VARS
    missing = [c for c in needed if c not in daily.columns]
    if missing:
        raise KeyError(f"Colunas faltando após resample: {missing}\nDisponíveis: {list(daily.columns)[:12]} ...")

    daily = daily.ffill()
    return daily

def series_for_coord(coord: Tuple[float, float], df_all_stations: pd.DataFrame) -> Optional[pd.DataFrame]:
    df = df_all_stations.copy()
    df["Distancia"] = df.apply(lambda r: calculate_distance_from_point_to_station(r, coord), axis=1)
    df_nearest = df[df["Distancia"] < RADIUS_KM]
    if df_nearest.empty:
        df_nearest = df.sort_values("Distancia").head(1)  # fallback: estação mais próxima
    return build_daily_panel_from_stations(df_nearest)

# =============================
# Exógenas com máscaras (sem vazamento)
# =============================
def _profiles_from_train(df_training: pd.DataFrame, exog_vars):
    prof_mmdd = (
        df_training.assign(__mm__=df_training.index.month, __dd__=df_training.index.day)
                   .groupby(["__mm__", "__dd__"])[exog_vars]
                   .median()
    )
    prof_doy = (
        df_training.assign(__doy__=df_training.index.dayofyear)
                   .groupby("__doy__")[exog_vars]
                   .median()
    )
    return prof_mmdd, prof_doy

def _impute_with_profiles(s: pd.Series, mmdd_index, doy_index, prof_mmdd, prof_doy, train_col_median):
    need = s.isna()
    if need.any():
        s.loc[need] = prof_mmdd[s.name].reindex(mmdd_index[need]).values
    need = s.isna()
    if need.any():
        s.loc[need] = prof_doy[s.name].reindex(doy_index[need]).values
    need = s.isna()
    if need.any():
        s.loc[need] = train_col_median
    return s

def prepare_exog_with_masks(df_training: pd.DataFrame,
                            df_test: pd.DataFrame,
                            exog_vars,
                            fourier_period: int = FOURIER_PERIOD,
                            fourier_k: int = FOURIER_K):
    """
    Retorna X_train, X_test, feature_names com:
      [exógenas imputadas | máscaras *_MISSING | Fourier]
    Máscaras: 1 quando o valor ORIGINAL era NaN, 0 caso contrário.
    Imputação do TESTE usa apenas estatísticas do TREINO (sem vazamento).
    """
    # máscaras originais
    train_masks = {f"{col}_MISSING": df_training[col].isna().astype(int) for col in exog_vars}
    test_masks  = {f"{col}_MISSING": df_test[col].isna().astype(int)      for col in exog_vars}

    prof_mmdd, prof_doy = _profiles_from_train(df_training, exog_vars)

    # índices auxiliares
    mmdd_tr = pd.MultiIndex.from_arrays([df_training.index.month, df_training.index.day], names=["__mm__", "__dd__"])
    mmdd_te = pd.MultiIndex.from_arrays([df_test.index.month,     df_test.index.day    ], names=["__mm__", "__dd__"])
    doy_tr  = ((df_training.index.dayofyear - 1) % 365) + 1
    doy_te  = ((df_test.index.dayofyear     - 1) % 365) + 1

    tr_imp = df_training[exog_vars].copy()
    te_imp = df_test[exog_vars].copy()

    # imputação treino
    for col in exog_vars:
        tr_imp[col] = _impute_with_profiles(
            tr_imp[col], mmdd_tr, doy_tr, prof_mmdd, prof_doy,
            train_col_median=df_training[col].median(skipna=True)
        )
    # imputação teste (sem vazamento)
    for col in exog_vars:
        te_imp[col] = _impute_with_profiles(
            te_imp[col], mmdd_te, doy_te, prof_mmdd, prof_doy,
            train_col_median=df_training[col].median(skipna=True)
        )

    # Fourier
    n_tr, n_te = len(df_training), len(df_test)
    t_tr = np.arange(n_tr)
    t_te = np.arange(n_tr, n_tr + n_te)
    F_tr = create_fourier_terms(t_tr, fourier_period, fourier_k)
    F_te = create_fourier_terms(t_te, fourier_period, fourier_k)
    fourier_cols = [f"F_{k}_{fn}" for k in range(1, fourier_k + 1) for fn in ("sin", "cos")]

    # monta X
    X_train = np.hstack([
        tr_imp.to_numpy(),
        np.column_stack([train_masks[m].to_numpy() for m in train_masks]),
        F_tr
    ])
    X_test  = np.hstack([
        te_imp.to_numpy(),
        np.column_stack([test_masks[m].to_numpy() for m in test_masks]),
        F_te
    ])

    feature_names = list(exog_vars) + list(train_masks.keys()) + fourier_cols
    return X_train, X_test, feature_names

# =============================
# ARIMA(3,0,1) com exógenas + Fourier + calibração
# =============================
def debias_with_calibration(model_fit, df_training, forecast_parameter, X_train_final, forecast_vals, calib_days=60):
    start = max(0, len(df_training) - calib_days)
    pred_cal = model_fit.get_prediction(
        start=df_training.index[start],
        end=df_training.index[-1],
        exog=X_train_final[start:]
    ).predicted_mean
    y_cal = df_training[forecast_parameter].iloc[start:]
    ME = (y_cal - pred_cal).mean()
    forecast_bias_fixed = forecast_vals + ME
    X = sm.add_constant(pred_cal.values)
    a, b = sm.OLS(y_cal.values, X).fit().params
    forecast_linear_cal = a + b * forecast_vals
    return {"ME": ME, "forecast_bias_fixed": forecast_bias_fixed, "a": a, "b": b, "forecast_linear_cal": forecast_linear_cal}

def arima_forecast_with_fourier_terms_exog(df_training: pd.DataFrame,
                                           df_test: pd.DataFrame,
                                           forecast_parameter: str,
                                           trend: str = "ct",
                                           calib_days: int = 60) -> pd.Series:
    X_train, X_test, _ = prepare_exog_with_masks(df_training, df_test, EXOG_VARS, FOURIER_PERIOD, FOURIER_K)

    model = ARIMA(df_training[forecast_parameter], exog=X_train, order=ARIMA_ORDER, trend=trend)
    model_fit = model.fit()

    forecast_vals = model_fit.forecast(steps=len(df_test), exog=X_test)
    cal = debias_with_calibration(model_fit, df_training, forecast_parameter, X_train, forecast_vals, calib_days=calib_days)

    forecast_corrected = pd.Series(cal["forecast_bias_fixed"], index=df_test.index, name="forecast")
    return forecast_corrected

# =============================
# Pipeline principal
# =============================
def run_for_coords(coords: List[Tuple[float, float]]) -> Dict[Tuple[float, float], Dict]:
    df_all = load_all_stations()
    results: Dict[Tuple[float, float], Dict] = {}

    for coord in coords:
        daily = series_for_coord(coord, df_all)
        if daily is None:
            results[coord] = {"ok": False, "reason": "Sem dados para essa coordenada."}
            continue

        split_idx = int(len(daily) * SPLIT_TRAIN)
        df_training = daily.iloc[:split_idx].copy()
        df_test     = daily.iloc[split_idx:].copy()

        # sanity check
        needed = [FORECAST_PARAMETER] + EXOG_VARS
        if any(col not in df_training.columns for col in needed) or any(col not in df_test.columns for col in needed):
            results[coord] = {"ok": False, "reason": "Colunas necessárias ausentes (target/exógenas)."}
            continue

        forecast = arima_forecast_with_fourier_terms_exog(df_training, df_test, FORECAST_PARAMETER)

        # métricas (RMSE com debias; MSE; sMAPE)
        mse = mean_squared_error(df_test[FORECAST_PARAMETER], forecast)
        rmse = mse ** 0.5
        smape_val = smape(df_test[FORECAST_PARAMETER], forecast)

        results[coord] = {
            "ok": True,
            "periodo": f"{daily.index.min().date()} → {daily.index.max().date()}",
            "n_total": len(daily),
            "n_train": len(df_training),
            "n_test": len(df_test),
            "RMSE": float(rmse),           # já com debias
            "MSE": float(mse),
            "sMAPE": float(smape_val),
            "forecast": forecast,          # série prevista (apenas janela de teste)
        }

    return results

# =============================
# Rodar para TODAS as universidades e imprimir tabela
# =============================
# Lê CSV das universidades
try:
    df_uni = pd.read_csv(UNI_CSV)  # tenta separador padrão
except Exception:
    df_uni = pd.read_csv(UNI_CSV, sep=";", decimal=",")  # fallback

df_uni.columns = [c.strip() for c in df_uni.columns]
assert {"Universidade", "Estado", "Latitude", "Longitude"}.issubset(df_uni.columns), \
    f"CSV de universidades não tem as colunas esperadas. Tenho: {df_uni.columns.tolist()}"

# Monta coords na mesma ordem do CSV
coords = [(float(lat), float(lon)) for lat, lon in zip(df_uni["Latitude"], df_uni["Longitude"])]

# Executa
results = run_for_coords(coords)

# Tabela final
rows = []
for i, coord in enumerate(coords):
    uni = df_uni.iloc[i]["Universidade"]
    uf  = df_uni.iloc[i]["Estado"]
    r   = results.get(coord, {})
    if r.get("ok"):
        rows.append({
            "Universidade": uni,
            "Estado": uf,
            "Latitude": round(coord[0], 6),
            "Longitude": round(coord[1], 6),
            "Período": r["periodo"],
            "Obs Totais": r["n_total"],
            "Treino": r["n_train"],
            "Teste": r["n_test"],
            "RMSE (debias)": round(r["RMSE"], 3),
            "MSE": round(r["MSE"], 3),
            "sMAPE (%)": round(r["sMAPE"], 3),
        })
    else:
        rows.append({
            "Universidade": uni,
            "Estado": uf,
            "Latitude": round(coord[0], 6),
            "Longitude": round(coord[1], 6),
            "Status": "Falhou",
            "Motivo": r.get("reason", "Sem dados após filtro/transformações"),
        })

df_resultados = pd.DataFrame(rows)
sort_cols = [c for c in ["Status", "Estado", "Universidade"] if c in df_resultados.columns]
df_resultados = df_resultados.sort_values(by=sort_cols).reset_index(drop=True)

print("\n=== RESULTADOS POR UNIVERSIDADE (ARIMA(3,0,1) + EXÓGENAS + FOURIER + MASKS) ===")
print(df_resultados)

# opcional: salvar
# df_resultados.to_csv("resultados_arima_universidades.csv", index=False, sep=";")
